# Создание словарей

Допустим, у нас будет всего 6 словарей: 
1. Словарь униграмм, где ключом будет униграмма, а значением - номер Q. 
2. Словарь биграмм, где ключом будет биграмма, а значением - номер Q. 
3. Словарь триграмм. 
4. Словарь четырехграмм. 
5. Словарь пятиграмм. 
6. На всякий случай: Общий обратный словарь, где ключ - это номер сущности Q, а значения - это все возможные наименования этой сущности в нашей системе словарей. Возможно, он пригодится (например, для контроля того, как нарезаются наши сущности). 

### Программа должна проходиться по чистому нормализованному датасету Викиданных и делать следующее:

    1.Создать общий обратный словарь, где номер Q - ключ, значения пока оставим пустыми. 
    2.Проверять длину каждого наименования сущности в каждой строке данных. 

- если это сущность длины 1, сразу кладем ее в словарь униграмм. Если такая сущность в словаре уже есть, прибавляем новый номер Q к сущности. То есть, одной сущности в словаре может соответствовать несколько номеров Q. 
- если это сущность длины 2, кладем ее в словарь биграмм, потом меняем элементы местами и снова кладем ее в словарь биграмм. Опять же добавляем номера Q, если вдруг ключи повторяются (на последующих этапах то же самое).  
- если это сущность длины 3, кладем ее в словарь триграмм. Также комбинаторно меняем слова сущности местами и кладем в словарь триграмм с другим ключом. А еще комбинаторно выбираем биграммы из триграмм и кладем в словарь биграмм. Это может быть полезно в случае имен собственных. 
- если это сущность длины 4, кладем ее в словарь тетраграмм, а также ее комбинаторные варианты. Также разбиваем (уже не комбинаторно) сущность на триграммы по порядку (1-2-3 слово, 2-3-4 слово) и кладем эти триграммы в словарь триграмм. 
- если это сущность длины 5, кладем ее в словарь пентаграмм, а также ее комбинаторные варианты. Также разбиваем сущность по порядку на триграммы и тетраграммы и кладем в соответствующие словари. 

    3.После того, как мы положили что-то в один из словарей, сразу добавляем это наименование сущности (или ее часть) в обратный словарь. 
    4.Программа должна вернуть 6 словарей соответственно. 

Возможно, стоит разбить эту программу на более маленькие части - например, создавать эти словари постепенно. 

In [ ]:
%%time

# сначала прочтем нормализованные Викиданные из файла
def read_tsv(file):
    with open(file, "r") as f:
        f = f.readlines()
        print(len(f))
    data = []
    for line in f:
        line = line.strip('\n').split(' | ')
        data.append(line)
    return data

norm_data = read_tsv("norm-wikidata.tsv")

2234352


In [2]:
norm_data[:10]

[['Q31', 'бельгия', 'королевство бельгия'],
 ['Q8', 'счастие'],
 ['Q23', 'джордж вашингтон', 'вашингтон джордж'],
 ['Q24', 'джек бауэр'],
 ['Q42', 'дуглас адамс', 'адамс дуглас'],
 ['Q1868', 'поль отля', 'отля поль'],
 ['Q2013', 'викидать', 'викидата'],
 ['Q45', 'португалия', 'португальский республика'],
 ['Q51', 'антарктида'],
 ['Q58', 'пенис', 'детородный орган', 'половый член', 'член']]

## Создаем функцию, которая обновляет целевой словарь и обратный словарь

In [7]:
def update_dict(dict_, ngrams, Q):
    
    if isinstance(ngrams, str):
        ngram = ngrams
        if ngram not in dict_:
            dict_[ngram] = {Q}
        else:
            dict_[ngram].add(Q)
        back_dict[Q].add(ngram)

    if isinstance(ngrams, list):
        for ngram in ngrams:
            if ngram not in dict_:
                dict_[ngram] = {Q}
            else:
                dict_[ngram].add(Q)
            back_dict[Q].add(ngram)
            
    return 0

## Создаем обратный словарь

In [4]:
back_dict = {line[0]:set() for line in norm_data}

In [5]:
len(back_dict)

2234352

## Создаем словарь униграмм (пока только для тех сущностей, у которых 1 название)

In [8]:
unigram_dict = {}
for line in norm_data:
    if len(line[1:]) == 1 and len(line[1].split()) == 1:
        Q, unigram = line[0], line[1]
        update_dict(unigram_dict, unigram, Q)
len(unigram_dict)

361549

In [9]:
unigram_dict["монпелье"]
# город во Франции
# округ Франции
# французский гандбольный клуб
# French women's association football club
# французский футбольный клуб из города Монпелье
# страница значений в Викиданных

{'Q19513', 'Q224482', 'Q3085161', 'Q6441', 'Q701804', 'Q901613'}

## Теперь будем проверять сущность на то, сколько у нее названий, и создавать другие словари

In [10]:
# еще unigram_dict
for line in norm_data:
    if len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name_split = entity_name.split()
            if len(entity_name_split) == 1:
                Q, unigram = line[0], entity_name_split[0]
                update_dict(unigram_dict, unigram, Q)

In [11]:
len(unigram_dict)

475538

## Биграммы

In [12]:
#bigram dict
bigram_dict = {}

for line in norm_data:
    if len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name = entity_name.split()
            if len(entity_name) == 2:
                bigrams = [' '.join(entity_name), ' '.join(entity_name[::-1])]
                Q = line[0]
                update_dict(bigram_dict, bigrams, Q)

In [13]:
len(bigram_dict)

1046937

In [14]:
back_dict["Q661"]

{'гидравлический таран',
 'гидротаранный насос',
 'насос гидротаранный',
 'таран гидравлический'}

## Триграммы

In [15]:
from itertools import permutations

trigram_dict = {}
for line in norm_data:
    if len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name = entity_name.split(' ')
            if len(entity_name) == 3:
                trigrams = [' '.join(word) for word in permutations(entity_name, 3)]
                bigrams = [' '.join(word) for word in permutations(entity_name, 2)]
                Q = line[0]
                update_dict(trigram_dict, trigrams, Q)
                update_dict(bigram_dict, bigrams, Q)
                    
print(len(trigram_dict))
print(len(bigram_dict))

2007943
2088445


In [16]:
back_dict["Q819"]

{'лаос',
 'лаосский народно-демократический',
 'лаосский народно-демократический республика',
 'лаосский республика',
 'лаосский республика народно-демократический',
 'народно-демократический лаосский',
 'народно-демократический лаосский республика',
 'народно-демократический республика',
 'народно-демократический республика лаосский',
 'республика лаосский',
 'республика лаосский народно-демократический',
 'республика народно-демократический',
 'республика народно-демократический лаосский'}

## Четырехграммы

In [17]:
fourgram_dict = {}

for line in norm_data:
    if len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name = entity_name.split()
            if len(entity_name) == 4:
                fourgrams = [' '.join(word) for word in permutations(entity_name, 4)]
                trigrams = [' '.join(entity_name[:-1]), ' '.join(entity_name[1:])]
                Q = line[0]  
                update_dict(fourgram_dict, fourgrams, Q)
                update_dict(trigram_dict, trigrams, Q)
                    
print(len(fourgram_dict))
print(len(trigram_dict))

1561959
2108718


## Пятиграммы

In [18]:
fivegram_dict = {}

for line in norm_data:
    if len(line[1:]) > 1:
        for entity_name in line[1:]:
            entity_name = entity_name.split()
            if len(entity_name) == 5:
                fivegrams = [' '.join(words) for words in permutations(entity_name, 5)]
                fourgrams = [' '.join(entity_name[:-1]), ' '.join(entity_name[1:])]
                trigrams = [' '.join(words) for words in [entity_name[:-2], entity_name[1:-1], entity_name[2:]]]
                Q = line[0]
                update_dict(fivegram_dict, fivegrams, Q)
                update_dict(fourgram_dict, fourgrams, Q)
                update_dict(trigram_dict, trigrams, Q)
                    
print(len(fivegram_dict))
print(len(fourgram_dict))
print(len(trigram_dict))

3391141
1606492
2150441


In [19]:
print(back_dict['Q11394'])
len(back_dict['Q11394'])

{'угроза вид подверженный вымирание биологический', 'вид исчезать', 'угроза подверженный биологический вымирание вид', 'вымирание подверженный биологический вид угроза', 'угроза вымирание подверженный вид биологический', 'вымирание угроза биологический вид подверженный', 'вымирание подверженный вид угроза биологический', 'угроза подверженный вымирание вид биологический', 'вымирание угроза вид подверженный биологический', 'вымирание вид биологический подверженный угроза', 'биологический вид подверженный вымирание угроза', 'вымирание вид угроза биологический подверженный', 'угроза подверженный биологический вид вымирание', 'вид вымирание угроза подверженный биологический', 'подверженный вымирание угроза вид биологический', 'подверженный вымирание вид биологический угроза', 'вымирание биологический угроза вид подверженный', 'угроза вид вымирание биологический подверженный', 'вид угроза подверженный вымирание биологический', 'угроза подверженный вид вымирание биологический', 'подверженный 

129

Многовато значений. Может, лучше не делать ключами все перестановки в четыреграммах и пятиграммах?  
С другой стороны, почему бы и нет? 